# Erstellen einer FHIR Procedure-Ressource

In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

In [1]:
from IPython.display import IFrame
import json
import requests
from fhirclient.models import (
    procedure,
    codeableconcept,
    coding,
    fhirreference,
    fhirdate
)

In [2]:
# Procedure Ressource in der FHIR Spezifikation.
IFrame('https://hl7.org/fhir/procedure.html', width=1200, height=330)

### Zur Orientierung bzgl. der verwendeten Codesysteme nutzen Sie das [Prozedur-Profil der Medizininformatik-Initiative](https://simplifier.net/medizininformatikinitiative-modulprozeduren/prozedur)

In [3]:
# Instanziieren des Prozedur-Objektes
my_procedure = procedure.Procedure()

In [4]:
# Setzen des Status auf 'completed' für eine abgeschlossene Prozedur
status = 'completed'
my_procedure.status = status

In [ ]:
# Vergeben eines Codes für die Prozedur aus dem Operationen- und Prozedurenschlüssel Version 2020
ops_code = codeableconcept.CodeableConcept()
ops_coding = coding.Coding()
ops_coding.system = 'http://fhir.de/CodeSystem/bfarm/ops'
ops_coding.version = '2020'
ops_coding.code = '5-470'
ops_coding.display = 'Appendektomie'
ops_code.coding = [ops_coding]
my_procedure.code = ops_code

### BONUS: Stellen Sie die Referenz zum zuvor hochgeladenen Patienten her

In [ ]:
fhir_test_server = 'https://server.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

response = requests.get(url=f'{fhir_test_server}/Patient?family=Holmes', headers=headers)

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
# Response Text als Python Dictionary holen
response_dict = json.loads(response.text)
response_dict

In [ ]:
# Referenz auf den Patienten
subject = fhirreference.FHIRReference()
pat_id = response_dict['entry'][0]['resource']['id']
subject.reference = f'Patient/{pat_id}'
my_procedure.subject = subject

In [ ]:
print(json.dumps(my_procedure.as_json(), indent=4))

## Aufgaben
1. Implementieren Sie den Durchführungszeitraum mit Start- und Endzeitpunkt der Prozedur (Tipp: zeitlicher Verlauf mit Start und Ende klingt nach Period!)
2. Lassen Sie die Ressource vom Server validieren und laden Sie diese in den Testserver hoch
------
Bonus: Stellen Sie die Referenz zum zuvor hochgeladenen Patienten her. Nutzen Sie hierfür die vorgestellten FHIR-Search Operatoren. 

### 1. Implementieren Sie den Durchführungszeitraum mit Start- und Endzeitpunkt der Prozedur (Tipp: zeitlicher Verlauf mit Start und Ende klingt nach Period!)

In [ ]:
from fhirclient.models import period

In [ ]:
performed_period = period.Period()

start = '2020-12-02T15:30:00+01:00'
end = '2020-12-02T16:15:00+01:00'

performed_period.start = fhirdate.FHIRDate(start)
performed_period.end = fhirdate.FHIRDate(end)

my_procedure.performedPeriod = performed_period

In [ ]:
print(json.dumps(my_procedure.as_json(), indent=4))

### 2. Lassen Sie die Ressource vom Server validieren und laden Sie diese in den Testserver hoch

In [ ]:
response = requests.post(f'{fhir_test_server}/Procedure/$validate', headers = headers, data = json.dumps(my_procedure.as_json()))
data = response.json()

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
print(json.dumps(data, indent=4))